In [ ]:
# factory reset runtime until you get p100
!nvidia-smi

Wed Apr 27 03:21:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
! pip install deeplabcut==2.2.0.6
import deeplabcut

In [ ]:
! cp -r /content/drive/MyDrive/5561/transfer/baseline ./baseline

In [ ]:
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Zips/train_cropped.zip train_cropped.zip
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Zips/val_cropped.zip val_cropped.zip
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Zips/test_cropped.zip test_cropped.zip
! cp /content/drive/MyDrive/5561/coco/train2017_cropped.zip ./human.zip

In [ ]:
%%capture
! unzip train_cropped.zip
! unzip val_cropped.zip
! unzip test_cropped.zip
! unzip human.zip

In [ ]:
import os
import shutil

num_train = 66917
num_val = 22306

for i in range(num_train):
  filename = f"train_cropped_{str(i).rjust(7, '0')}.jpg"
  shutil.move("train_cropped/" + filename, "baseline/labeled-data/baseline/" + filename)

for i in range(num_val):
  filename = f"val_cropped_{str(i).rjust(7, '0')}.jpg"
  shutil.move("val_cropped/" + filename, "baseline/labeled-data/baseline/" + filename)

In [ ]:
# move human images into folder

for d in os.listdir("train2017_cropped"):
  source = f"train2017_cropped/{d}"
  dest = f"baseline/labeled-data/baseline/{d}"
  shutil.move(source, dest)

In [ ]:
path_config_file = "/content/baseline/config.yaml"

deeplabcut.create_training_dataset(path_config_file, augmenter_type="imgaug")

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([107699,  28653,  53866, ...,  47557, 102981,  79954]),
   array([95920,  3780, 24665, ..., 80150, 95812, 24599])))]

In [ ]:
shuffle=1
trainingsetindex=0

#get path to the model configuration files
trainposeconfigfile,testposeconfigfile,snapshotfolder=deeplabcut.return_train_network_path(path_config_file, shuffle=shuffle ,trainingsetindex=trainingsetindex)

#load the training config file and change various parameters:
cfg_dlc=deeplabcut.auxiliaryfunctions.read_plainconfig(trainposeconfigfile)

cfg_dlc['multi_step']=[[0.001, 1030000]]
cfg_dlc["init_weights"] = "/content/baseline/dlc-models/iteration-0/baselineMar19-trainset95shuffle1/train/snapshot-500000"

#save it and then train the model!
deeplabcut.auxiliaryfunctions.write_plainconfig(trainposeconfigfile,cfg_dlc)

In [ ]:

# train
deeplabcut.train_network(path_config_file, max_snapshots_to_keep=5, displayiters=1000, saveiters=10000, maxiters=1030000, allow_growth=False)

# Save model weights
! zip drive/MyDrive/5561/human-snapshots/snapshot-1000000.zip /content/baseline/dlc-models/iteration-0/baselineMar19-trainset95shuffle1/train/snapshot-1000000*


# inference
# deeplabcut.analyze_videos(path_config_file,
#                           [f'/content/test_cropped'], 
#                           videotype='.jpg', 
#                           destfolder=f"/content/test_predictions")

# # save results
# shutil.make_archive(f"/content/drive/MyDrive/5561/regular-occlusion-predictions/test_predictions50", 'zip', f"/content/test_predictions")

In [ ]:
while True:
  pass

In [ ]:
! zip drive/MyDrive/5561/regular-snapshots-50/snapshot-90000.zip /content/baseline/dlc-models/iteration-0/baselineMar19-trainset95shuffle1/train/snapshot-90000*